In [ ]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import pandas as pd
import numpy as np

In [ ]:
class CodeClassifier:
    """
    Multi-class classifier wrapper.
    Supports Logistic Regression, Decision Tree,
    Random Forest, and SVM.
    """
    def __init__(self, model_name='logistic', **kwargs):
        models = {
            'logistic': LogisticRegression,
            'decision_tree': DecisionTreeClassifier,
            'random_forest': RandomForestClassifier,
            'svm': SVC
        }
        if model_name not in models:
            raise ValueError(f"Model '{model_name}' not supported.")
        self.model = models[model_name](**kwargs)

    def fit(self, X, y):
        """Fit the model."""
        self.model.fit(X, y)

    def predict(self, X):
        """Predict labels."""
        return self.model.predict(X)

    def evaluate(self, X, y):
        """
        Compute accuracy, precision, recall, and F1 score.
        Uses macro-average for multi-class.
        """
        y_pred = self.predict(X)
        return {
            'accuracy': accuracy_score(y, y_pred),
            'precision': precision_score(y, y_pred, average='macro', zero_division=0),
            'recall': recall_score(y, y_pred, average='macro', zero_division=0),
            'f1_score': f1_score(y, y_pred, average='macro', zero_division=0)
        }



In [ ]:
df = pd.read_csv("Complexity10features.csv")
df.head()

Sentences       TTR        MLTD  \
0  LOS ANGELES, June 17 (AFP) Supermodel Rachel H...  0.722727  121.546961   
1  To evaluate ADAT, we introduce MedASL, the fir...  0.605263   60.823171   
2  ANKARA, Turkey (AP) An explosion in a cafe at ...  0.562791   56.240658   
3  At the beginning of the 7th century of our era...  0.666667   37.500000   
4  In this paper, we attack the problem of parsin...  0.587571   59.572636   

    Entropy  AvgWordLength  HapaxRatio  AvgParseDepth  MaxParseDepth  \
0  7.042371       4.786364    0.586364       7.125000             11   
1  5.818427       4.456140    0.412281       7.250000              8   
2  6.295037       4.716279    0.409302       6.416667              9   
3  5.724355       4.852941    0.519608      10.000000             13   
4  6.266801       5.005650    0.423729       8.833333             13   

   SubordClauseRatio  CenterEmbedDepth  AvgDepDistance  RecDepChainLen  \
0           0.692308                 1        3.531250               4   
1           0.428571                 0        3.604478               0   
2           0.478261                 1        3.231111               2   
3           0.727273                 3        3.831858               1   
4           0.500000                 0        3.684211               1   

   AvgTUnitLength  ClausePerSentRatio  SynStructVar  RecSynDepth  Labels  
0       26.250000            3.250000      1.713914            4       3  
1       23.500000            1.750000      0.829156            2       6  
2       17.750000            1.916667      1.187317            2       3  
3       33.666667            3.666667      2.054805            1       5  
4       29.000000            2.000000      0.577350            1       4

In [ ]:
# 2. Define feature columns and label
#feature_cols = [
    # "TTR", "MLTD", "Entropy", "AvgWordLength", "HapaxRatio",
    # "AvgParseDepth", "MaxParseDepth", "SubordClauseRatio",
    # "CenterEmbedDepth", "AvgDepDistance", "RecDepChainLen",
    # "AvgTUnitLength", "ClausePerSentRatio", "SynStructVar", "RecSynDepth"]


#feature_cols = ['TTR', 'AvgDepDistance', 'RecDepChainLen', 'SynStructVar', 'RecSynDepth']

feature_cols = [
    "TTR",
    "MLTD",
    "Entropy",
    "AvgWordLength",
    "HapaxRatio",
    "AvgParseDepth",
    "MaxParseDepth",
    "SubordClauseRatio",
    "CenterEmbedDepth",
    "AvgDepDistance",
    "RecDepChainLen",
    "AvgTUnitLength",
    "ClausePerSentRatio",
    "SynStructVar",
    "RecSynDepth"
]

X = df[feature_cols]
y = df["Labels"]

# 3. Scale features with Min-Max scaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 4. Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

In [ ]:
# List of models to compare
models = ['logistic', 'decision_tree', 'random_forest', 'svm']
results = []

# Loop through each model, train, and evaluate
for model_name in models:
    clf = CodeClassifier(model_name=model_name, random_state=42)
    clf.fit(X_train, y_train)
    m = clf.evaluate(X_test, y_test)
    m['Model'] = model_name
    results.append(m)

# Create a DataFrame of results
df_results = pd.DataFrame(results)[['Model', 'accuracy', 'precision', 'recall', 'f1_score']]

In [ ]:
clf = CodeClassifier(model_name='decision_tree', random_state=42)
clf.fit(X_train, y_train)
metrics = clf.evaluate(X_test, y_test)

print("Evaluation Metrics on Provided Data:")
for name, val in metrics.items():
    print(f"{name.capitalize()}: {val:.4f}")

Evaluation Metrics on Provided Data:
Accuracy: 0.3871
Precision: 0.3141
Recall: 0.3095
F1_score: 0.3031


In [ ]:
df_results

Model  accuracy  precision    recall  f1_score
0       logistic  0.322581   0.241667  0.201190  0.162067
1  decision_tree  0.387097   0.314069  0.309524  0.303061
2  random_forest  0.354839   0.215306  0.251190  0.226551
3            svm  0.322581   0.241667  0.201190  0.162067

In [ ]:
clf.model.feature_importances_

array([0.03150607, 0.09485816, 0.20091457, 0.07690168, 0.        ,
       0.02876281, 0.02364066, 0.04255319, 0.03191489, 0.03152088,
       0.        , 0.22162262, 0.08133339, 0.13447108, 0.        ])

In [ ]:
importances = clf.model.feature_importances_
features = feature_cols  # your list of feature names
for feat, imp in sorted(zip(features, importances), key=lambda x: -x[1])[:10]:
    print(f"{feat}")

AvgTUnitLength
Entropy
SynStructVar
MLTD
ClausePerSentRatio
AvgWordLength
SubordClauseRatio
CenterEmbedDepth
AvgDepDistance
TTR


In [ ]:
read = pd.read_csv("full_event_complexity.csv")
read.head()

FileNotFoundError: [Errno 2] No such file or directory: 'full_event_complexity.csv'

In [ ]:
# 1. Define your feature columns
feature_cols = [
    "TTR", "MLTD", "Entropy", "AvgWordLength", "HapaxRatio",
    "AvgParseDepth", "MaxParseDepth", "SubordClauseRatio",
    "CenterEmbedDepth", "AvgDepDistance", "RecDepChainLen",
    "AvgTUnitLength", "ClausePerSentRatio", "SynStructVar",
    "RecSynDepth"
]

# 2. Build the feature matrix from df1
X_pred = read[feature_cols]

# 3. Predict with your already-trained classifier
preds = clf.predict(X_pred)

# 4. Attach the results back to df1
read["predicted_label"] = preds

NameError: name 'read' is not defined

In [ ]:
read = read[["event_text", "predicted_label"]]
read.head()

event_text  predicted_label
0  Event schemas are a form of world knowledge ab...                8
1  In contrast, we propose to treat event schemas...                8
2  Compared to directly using LLMs to generate a ...                8
3  Large language models (LLMs) have been widely ...                8
4  We design a constrained keywords-to-sentence g...                8

In [ ]:
df1 = pd.read_csv("Readability10features.csv")
df1.head()

Sentence  avg_word_length  \
0  LOS ANGELES, June 17 (AFP) Supermodel Rachel H...         4.928571   
1  To evaluate ADAT, we introduce MedASL, the fir...         4.914894   
2  ANKARA, Turkey (AP) An explosion in a cafe at ...         4.751174   
3  At the beginning of the 7th century of our era...         4.871287   
4  In this paper, we attack the problem of parsin...         5.000000   

   avg_syllables  avg_age_of_acquisition  avg_word_familiarity  \
0       1.633333                       8                     3   
1       1.861702                       8                     3   
2       1.732394                       8                     3   
3       1.673267                       8                     3   
4       1.660920                       8                     3   

   flesch_reading_ease  gunning_fog  lexical_overlap  connective_density  \
0                53.04        15.32         0.083100            0.000000   
1                55.81         8.49         0.198041            0.000000   
2                51.78        13.21         0.088556            0.000000   
3                28.81        21.80         0.107915            0.009901   
4                33.78        16.38         0.123057            0.000000   

   referential_cohesion  semantic_similarity  avg_cosine_similarity  \
0              0.375000             0.263809               0.263809   
1              0.500000             0.587936               0.587936   
2              0.000000             0.214002               0.214002   
3              0.333333             0.292178               0.292178   
4              0.000000             0.414186               0.414186   

   variance_of_embedding_distances  Labels  
0                         0.020938       7  
1                         0.011793       4  
2                         0.030197       7  
3                         0.007357       6  
4                         0.049934       6

In [ ]:
# feature_cols = ['avg_syllables', 'gunning_fog',
# 'lexical_overlap', 'connective_density',
# 'referential_cohesion', 'variance_of_embedding_distances']

In [ ]:
feature_cols = [
    "avg_word_length",
    "avg_syllables",
    "avg_age_of_acquisition",
    "avg_word_familiarity",
    "flesch_reading_ease",
    "gunning_fog",
    "lexical_overlap",
    "connective_density",
    "referential_cohesion",
    "semantic_similarity",
    "avg_cosine_similarity",
    "variance_of_embedding_distances"
]


In [ ]:
#feature_cols = [col for col in df1.columns if col not in ['Paragraph', 'Labels']]
X = df1[feature_cols].astype(str)
y = df1['Labels']

# 3. Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 4. Train

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)



In [ ]:
# 6. Compare all models
# List of models to compare
models = ['logistic', 'decision_tree', 'random_forest', 'svm']
results = []

# Loop through each model, train, and evaluate
for model_name in models:
    clf = CodeClassifier(model_name=model_name, random_state=42)
    clf.fit(X_train, y_train)
    m = clf.evaluate(X_test, y_test)
    m['Model'] = model_name
    results.append(m)

# Create a DataFrame of results
df_results = pd.DataFrame(results)[['Model', 'accuracy', 'precision', 'recall', 'f1_score']]

In [ ]:
df_results

Model  accuracy  precision    recall  f1_score
0       logistic  0.387097   0.233333  0.391608  0.227035
1  decision_tree  0.225806   0.154762  0.252914  0.145175
2  random_forest  0.483871   0.227778  0.364802  0.241092
3            svm  0.516129   0.324242  0.458741  0.326475

In [ ]:
metrics

{'accuracy': 0.3870967741935484,
 'precision': 0.31406926406926405,
 'recall': 0.3095238095238095,
 'f1_score': 0.30306122448979594}

In [ ]:
# 5. Test
clf = CodeClassifier(model_name='random_forest', random_state=42)
clf.fit(X_train, y_train)
metrics = clf.evaluate(X_test, y_test)

In [ ]:
#clf.model.

<bound method BaseEstimator.get_params of SVC(random_state=42)>

In [ ]:
metrics

{'accuracy': 0.5161290322580645,
 'precision': 0.3242424242424242,
 'recall': 0.45874125874125876,
 'f1_score': 0.32647527910685803}

In [ ]:
importances = clf.model.feature_importances_
features = feature_cols  # your list of feature names
for feat, imp in sorted(zip(features, importances), key=lambda x: -x[1])[:10]:
    print(f"{feat}")

avg_word_length
variance_of_embedding_distances
lexical_overlap
avg_syllables
semantic_similarity
avg_cosine_similarity
flesch_reading_ease
connective_density
gunning_fog
referential_cohesion


In [ ]:
read2 = pd.read_csv("full_event_readability.csv")
read2.head()

event_text  avg_word_length  \
0  Event schemas are a form of world knowledge ab...         5.422222   
1  In contrast, we propose to treat event schemas...         5.693182   
2  Compared to directly using LLMs to generate a ...         5.464789   
3  Large language models (LLMs) have been widely ...         5.206897   
4  We design a constrained keywords-to-sentence g...         4.950000   

   avg_syllables  avg_age_of_acquisition  avg_word_familiarity  \
0       1.866667                     8.0                   3.0   
1       1.897727                     8.0                   3.0   
2       1.957746                     8.0                   3.0   
3       2.000000                     8.0                   3.0   
4       1.700000                     8.0                   3.0   

   flesch_reading_ease  gunning_fog  lexical_overlap  connective_density  \
0                31.72        16.11         0.114286            0.000000   
1                25.12        18.04         0.074907            0.000000   
2                39.74        12.96         0.096774            0.000000   
3                48.30        11.32         0.101317            0.017241   
4                44.75        13.87         0.000000            0.000000   

   referential_cohesion  semantic_similarity  avg_cosine_similarity  \
0                   0.0             0.715229               0.715229   
1                   0.0             0.607764               0.607764   
2                   0.0             0.394734               0.394734   
3                   0.0             0.534439               0.534439   
4                   0.0             0.000000               0.000000   

   variance_of_embedding_distances  
0                         0.000000  
1                         0.002553  
2                         0.000000  
3                         0.013912  
4                         0.000000

In [ ]:
# 2. Build the feature matrix from df1
X_pred = read2[feature_cols]

# 3. Predict with your already-trained classifier
preds = clf.predict(X_pred)

# 4. Attach the results back to df1
read2["predicted_label"] = preds

# 5. Quick check
#print(df2[["event_Text", "predicted_label"]].head())

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [ ]:
read2 = read2[["event_text", "predicted_label"]]

In [ ]:
read2

event_text  predicted_label
0     Event schemas are a form of world knowledge ab...                7
1     In contrast, we propose to treat event schemas...                7
2     Compared to directly using LLMs to generate a ...                7
3     Large language models (LLMs) have been widely ...                7
4     We design a constrained keywords-to-sentence g...                7
...                                                 ...              ...
1595  The research highlights the complex challenges...                7
1596  Ecological momentary assessment (EMA) is widel...                7
1597  We leveraged the availability of comprehensive...                7
1598  In unadjusted logistic regression models, gend...                7
1599  Results suggest that migration background is t...                7

[1600 rows x 2 columns]

In [ ]:
merged = pd.merge(
    read.rename(columns={'predicted_label': 'complexity_label'}),
    read2.rename(columns={'predicted_label': 'readability_label'}),
    on='event_text',
    how='inner'
)

In [ ]:
merged.to_csv("merged_labels.csv", index=False)